<a href="https://colab.research.google.com/github/vidisha02/SOC_GPT/blob/main/week3_22b1218.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [ ]:
words=open('names.txt','r').read().splitlines()

In [ ]:
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
print(itos)
vocab_size = len(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
blocksize=2
def build_dataset(words):

    X,Y=[],[]
    for w in words:
    # print(w)
        context=[0]* blocksize
        for ch in w+'.':
            ix=stoi[ch]
            X.append(context)
            Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context=context[1:]+[ix]

    X=torch.tensor(X)
    Y=torch.tensor(Y)
    return X,Y
import random
random.shuffle(words)
n1=int(0.8*len(words))
n2=int(0.9*len(words))

Xtr,Ytr=build_dataset(words[:n1])
Xdev,Ydev=build_dataset(words[n1:n2])
Xte,Yte=build_dataset(words[n2:])

In [ ]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
W1 = torch.randn((n_embd * blocksize, n_hidden), generator=g) *(5/3)/((n_embd*blocksize)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) *0.01
W2 = torch.randn((n_hidden, vocab_size),          generator=g) *0.01
b2 = torch.randn(vocab_size,                      generator=g) *0
bngain=torch.ones((1,n_hidden))
bnbias=torch.zeros((1,n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2,bngain,bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

10097


In [ ]:
lossi=[]
batch_size = 80
max_steps = 200000
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  #print(embcat)
  # Linear layer
  hpreact = embcat @ W1 + b1
  #print(hpreact)
  #batch norm
  bnmeani = hpreact.mean(0, keepdim=True)
  bnstdi = hpreact.std(0, keepdim=True)+ 1e-5
  hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
 # print(bnstdi)
  with torch.no_grad():
    bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
    bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
  # Non-linearity
  h = torch.tanh(hpreact)
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function
  for p in parameters:
    p.grad = None
  loss.backward()
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')

  lossi.append(loss.log10().item())


      0/ 200000: 3.2964
  10000/ 200000: 2.8814
  20000/ 200000: 2.8036
  30000/ 200000: 2.7474
  40000/ 200000: 2.8232
  50000/ 200000: 2.7848
  60000/ 200000: 2.9411
  70000/ 200000: 2.9307
  80000/ 200000: 2.8364
  90000/ 200000: 2.9105
 100000/ 200000: 2.8326
 110000/ 200000: 2.9614
 120000/ 200000: 2.7053
 130000/ 200000: 2.6867
 140000/ 200000: 2.7778
 150000/ 200000: 2.7851
 160000/ 200000: 2.8678
 170000/ 200000: 2.7610
 180000/ 200000: 2.8501
 190000/ 200000: 2.7544


In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 # + b1
  # measure the mean/std over the entire training set
  bnmeani = hpreact.mean(0, keepdim=True)
  bnstdi = hpreact.std(0, keepdim=True)

In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1  + b1

  hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.8361928462982178
val 2.829237461090088


In [ ]:
bnmean_running

tensor([[-1.7790e-04,  9.3513e-05,  5.9275e-04,  8.5744e-04,  2.9018e-03,
          2.4782e-03,  8.6591e-04, -8.3598e-04, -1.6511e-03, -1.2940e-04,
         -8.7074e-04, -9.2555e-04, -1.1260e-04, -8.0415e-04, -5.6077e-04,
         -1.8292e-04, -1.2400e-04,  6.6950e-04,  3.4143e-04,  2.3904e-03,
          5.9338e-04, -1.0385e-03, -6.3765e-04,  2.3686e-03, -5.5471e-05,
          2.5760e-05, -1.5510e-03,  1.5278e-03, -1.3791e-03,  7.8750e-04,
         -3.5192e-04, -1.6714e-03,  1.3393e-03,  1.7677e-03, -2.5129e-03,
         -1.6825e-03, -9.7389e-04, -2.5741e-03,  9.2010e-04,  1.2929e-03,
         -1.4243e-03, -1.4036e-03, -1.1524e-03, -2.8446e-03,  6.7485e-04,
          4.0156e-04, -5.6678e-04, -9.9535e-04,  7.5596e-04,  2.0262e-03,
          8.5607e-06,  2.5890e-04,  1.0926e-03,  4.6882e-04,  1.0670e-03,
         -1.8383e-03,  1.0378e-04,  3.7025e-04,  3.1065e-03, -1.1766e-04,
          4.7767e-04, -6.3792e-04, -4.0523e-04,  1.4112e-03,  1.4460e-03,
          1.2434e-03,  9.0810e-04,  5.

In [ ]:
bnmeani

tensor([[-1.7790e-01,  9.3513e-02,  5.9275e-01,  8.5744e-01,  2.9018e+00,
          2.4782e+00,  8.6591e-01, -8.3598e-01, -1.6511e+00, -1.2940e-01,
         -8.7074e-01, -9.2555e-01, -1.1260e-01, -8.0415e-01, -5.6077e-01,
         -1.8292e-01, -1.2400e-01,  6.6950e-01,  3.4143e-01,  2.3904e+00,
          5.9338e-01, -1.0385e+00, -6.3765e-01,  2.3686e+00, -5.5471e-02,
          2.5760e-02, -1.5510e+00,  1.5278e+00, -1.3791e+00,  7.8750e-01,
         -3.5192e-01, -1.6714e+00,  1.3393e+00,  1.7677e+00, -2.5129e+00,
         -1.6825e+00, -9.7389e-01, -2.5741e+00,  9.2010e-01,  1.2929e+00,
         -1.4243e+00, -1.4036e+00, -1.1524e+00, -2.8446e+00,  6.7485e-01,
          4.0156e-01, -5.6678e-01, -9.9535e-01,  7.5596e-01,  2.0262e+00,
          8.5607e-03,  2.5890e-01,  1.0926e+00,  4.6882e-01,  1.0670e+00,
         -1.8383e+00,  1.0378e-01,  3.7025e-01,  3.1065e+00, -1.1766e-01,
          4.7767e-01, -6.3792e-01, -4.0523e-01,  1.4112e+00,  1.4460e+00,
          1.2434e+00,  9.0810e-01,  5.

In [ ]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
W1 = torch.randn((n_embd * blocksize, n_hidden), generator=g)  *0
b1 = torch.randn(n_hidden,                        generator=g) *0
W2 = torch.randn((n_hidden, vocab_size),          generator=g) *0
b2 = torch.randn(vocab_size,                      generator=g) *0


parameters = [C, W1, W2, b2]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

9697


In [ ]:
lossi=[]
batch_size = 70
max_steps = 200000
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hpreact = embcat @ W1 + b1


  # Non-linearity
  h = torch.tanh(hpreact)
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function
  for p in parameters:
    p.grad = None
  loss.backward()
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')

  lossi.append(loss.log10().item())

      0/ 200000: 3.2958
  10000/ 200000: 2.7389
  20000/ 200000: 2.7811
  30000/ 200000: 2.8257
  40000/ 200000: 2.8943
  50000/ 200000: 2.7639
  60000/ 200000: 2.9700
  70000/ 200000: 2.6942
  80000/ 200000: 2.8579
  90000/ 200000: 2.7909
 100000/ 200000: 2.8550
 110000/ 200000: 2.7052
 120000/ 200000: 2.7942
 130000/ 200000: 2.7334
 140000/ 200000: 2.8065
 150000/ 200000: 2.8491
 160000/ 200000: 2.9079
 170000/ 200000: 2.7299
 180000/ 200000: 2.8660
 190000/ 200000: 2.8075


In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1  + b1

  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.823745012283325
val 2.8260042667388916


In [ ]:
class Linear:

  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []


In [ ]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility

In [ ]:
C = torch.randn((vocab_size, n_embd),            generator=g)
layers = [
  Linear(n_embd * blocksize, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]
with torch.no_grad():
  layers[-1].gamma*=0.1
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 1.0

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

15424


In [ ]:
max_steps = 200000
batch_size = 100
lossi = []

for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  x = emb.view(emb.shape[0], -1) # concatenate the vectors
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, Yb) # loss function

  # backward pass
  for layer in layers:
    layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')

  lossi.append(loss.log10().item())


      0/ 200000: 3.1877
  10000/ 200000: 2.8221
  20000/ 200000: 2.8222
  30000/ 200000: 3.0149
  40000/ 200000: 2.7379
  50000/ 200000: 2.7921
  60000/ 200000: 2.8072
  70000/ 200000: 2.7852
  80000/ 200000: 2.8211
  90000/ 200000: 2.9024
 100000/ 200000: 2.8545
 110000/ 200000: 2.8069
 120000/ 200000: 2.7154
 130000/ 200000: 2.8438
 140000/ 200000: 2.7213
 150000/ 200000: 2.9383
 160000/ 200000: 2.8471
 170000/ 200000: 2.8159
 180000/ 200000: 2.9120
 190000/ 200000: 2.8617


In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  x = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, y)
  print(split, loss.item())

# put layers into eval mode
for layer in layers:
  layer.training = False
split_loss('train')
split_loss('val')

train 3.2005159854888916
val 3.2006030082702637
